<a href="https://colab.research.google.com/github/krishnamohanathota/GenerativeAI/blob/main/langchain/modules/L3_Chains.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Chains in LangChain

Using an LLM in isolation is fine for simple applications, but more complex applications require **chaining** LLMs

Chains allow us to combine multiple components together to create a single, coherent application.

For example, we can create a chain that takes user input, formats it with a PromptTemplate, and then passes the formatted response to an LLM.

In [1]:
!pip install openai
!pip install python-dotenv
!pip install langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 24.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 kB 6.0 MB/s eta 0:00:00


In [2]:
import os
import openai

from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())#Load local .env file

openai.api_key = os.environ['OPENAI_API_KEY']

## LLM Chain

https://python.langchain.com/docs/modules/chains/foundational/llm_chain

- LLMChain is a simple chain that adds some functionality around language models
- LLMChain consists of a PromptTemplate and a language model (either an LLM or chat model). It formats the prompt template using the input key values provided (and also memory key values, if available), passes the formatted string to LLM and returns the LLM output.

In [3]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate

llm = OpenAI(temperature=0.0)
prompt = PromptTemplate(input_variables=["product"],
                        template="What is a good name for a company that makes {product}?")
print(prompt)

#We can now create a very simple chain that will take user input, format the prompt with it, and then send it to the LLM.
from langchain.chains import LLMChain
chain = LLMChain(llm=llm, prompt=prompt)

# Run the chain only specifying the input variable.
product = "Cricket Bats"
print(chain.run(product))

input_variables=['product'] output_parser=None partial_variables={} template='What is a good name for a company that makes {product}?' template_format='f-string' validate_template=True


Cricket Bat Crafters.


In [4]:
#If there are multiple variables, you can input them all at once using a dictionary.
prompt = PromptTemplate(input_variables=["company", "product"],
                        template="What is a good name for a {company} that makes {product}?")

print(prompt)

from langchain.chains import LLMChain
chain = LLMChain(llm=llm, prompt=prompt)

imputDict = {'company' : 'ABC Company', "product" : "Shoe"}
print(chain.run(imputDict))

input_variables=['company', 'product'] output_parser=None partial_variables={} template='What is a good name for a {company} that makes {product}?' template_format='f-string' validate_template=True


FootFashionz.


In [7]:
# "Chat model" in an LLMChain

from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

prompt_template = ChatPromptTemplate.from_template("What is the best name to describe \
                                                    a company that makes {product}")

llm_chatmodel = ChatOpenAI(temperature=0.9)

chain = LLMChain(llm=llm_chatmodel, prompt=prompt_template)
#chain = LLMChain(llm=llm, prompt=prompt_template)

product = "bags"
print(chain.run(product))

The best name to describe a company that makes bags can vary depending on the specific focus or niche of the company. However, some possible options could be:

1. BagCrafters
2. BagMakers
3. SturdyCarry
4. StylePak
5. TrendyTotes
6. EliteCarriers
7. BagGenius
8. SecureSatchels
9. FashionJunction
10. VersaPack

Ultimately, the name should reflect the brand identity, target market, and desired perception of the company making bags.


## SimpleSequentialChain

It is a type of `sequential chain` that allows you to connect multiple chains and execute them in a specific order.

The simplest form of `sequential chains`, where each step has a singular input/output, and the output of one step is the input to the next.

It takes care of passing the output of one chain as the input to the next chain, allowing you to easily chain multiple steps together.

https://python.langchain.com/docs/modules/chains/foundational/sequential_chains

In [10]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import SimpleSequentialChain
from langchain.prompts import ChatPromptTemplate

In [14]:
llm = ChatOpenAI(temperature=0.0)

# prompt template 1
first_prompt = ChatPromptTemplate.from_template("What is the best name to describe \
    a company that makes {product}?")

# Chain 1
chain_one = LLMChain(llm=llm, prompt=first_prompt)

# prompt template 2
second_prompt = ChatPromptTemplate.from_template(
    "Write a 20 words description for the following \
    company:{company_name}"
)
# Chain 2
chain_two = LLMChain(llm=llm, prompt=second_prompt)

overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two],
                                             verbose=True
                                            )

product = "Cricket bats"
overall_simple_chain.run(product)



> Entering new SimpleSequentialChain chain...
"Cricket Bat Co."
Cricket Bat Co. is a leading manufacturer of high-quality cricket bats, offering exceptional performance and durability for players of all levels.

> Finished chain.


'Cricket Bat Co. is a leading manufacturer of high-quality cricket bats, offering exceptional performance and durability for players of all levels.'

## Sequential Chain

https://python.langchain.com/docs/modules/chains/foundational/sequential_chains#sequential-chain

Not all sequential chains will be as simple as passing a single string as an argument and getting a single string as output for all steps in the chain.

How we name the input/output variable names?  In the above example we didn't have to think about that because we were just passing the output of one chain directly as input to the next.

In [15]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import SequentialChain
from langchain.prompts import PromptTemplate

In [19]:
llm = ChatOpenAI(temperature=0.0)

# First Prompt
first_prompt = PromptTemplate.from_template("Translate the following review to english:"
    "\n\n{Review}")

# First Chain : input= Review and output= English_Review
chain_one = LLMChain(llm = llm, prompt = first_prompt, output_key = "English_Review")

# Second Prompt
second_prompt = PromptTemplate.from_template("Can you summarize the following review in 1 sentence:"
    "\n\n{English_Review}")

# Second Chain : input = English_Review and output = summary
chain_two = LLMChain(llm = llm, prompt=second_prompt, output_key="summary")

# Third Prompt
third_prompt = PromptTemplate.from_template("What language is the follwing review : \n \n {Review}")

#Third Chain : input = Review and output = language
chain_three = LLMChain(llm=llm, prompt=third_prompt, output_key="language")

review = "Le nouvel iPhone 14 Plus dispose d'un écran Super Retina XDR surdimensionné. La plus longue durée de batterie jamais vue. Un nouvel appareil photo principal et un traitement d'image amélioré vous permettent de capturer des photos encore plus sensationnelles dans toutes sortes de conditions lumineuses, en particulier en basse lumière. Que vous filmiez en randonnée sur un sentier rocheux ou que vous couriez après vos enfants dans le parc, essayez le mode Action pour des vidéos stables à main levée. Des fonctionnalités de sécurité, y compris l'appel d'urgence SOS via satellite et la détection d'accident, demandent de l'aide lorsque vous en avez besoin."

#Sequential Chain
sequential_chain = SequentialChain(chains=[chain_one, chain_two, chain_three],
                                   input_variables=["Review"],
                                   output_variables=["English_Review", "summary", "language"],
                                   verbose=True)


sequential_chain.run(review)


ValueError: ignored

In [20]:
sequential_chain(review)



> Entering new SequentialChain chain...

> Finished chain.


{'Review': "Le nouvel iPhone 14 Plus dispose d'un écran Super Retina XDR surdimensionné. La plus longue durée de batterie jamais vue. Un nouvel appareil photo principal et un traitement d'image amélioré vous permettent de capturer des photos encore plus sensationnelles dans toutes sortes de conditions lumineuses, en particulier en basse lumière. Que vous filmiez en randonnée sur un sentier rocheux ou que vous couriez après vos enfants dans le parc, essayez le mode Action pour des vidéos stables à main levée. Des fonctionnalités de sécurité, y compris l'appel d'urgence SOS via satellite et la détection d'accident, demandent de l'aide lorsque vous en avez besoin.",
 'English_Review': "The new iPhone 14 Plus features an oversized Super Retina XDR display. The longest battery life ever seen. A new main camera and improved image processing allow you to capture even more stunning photos in all kinds of lighting conditions, especially in low light. Whether you're filming while hiking on a roc

## Router Chain

https://python.langchain.com/docs/modules/chains/foundational/router

To create a chain that `dynamically` selects the next chain to use for a given input.

Router chains are made up of two components:

- The RouterChain itself (responsible for selecting the next chain to call)
- destination_chains: chains that the router chain can route to

--------------------

So far we've covered the `LLM chain` and then a `sequential chain`.

But what if you want to do something more complicated?

A pretty common but basic operation is to route an input to a chain depending on what exactly that `input` is.

A good way to imagine this is if you have multiple sub chains,
each of which specialized for a particular type of input,
you could have a router chain which first
decides which subchain to pass it to and then passes it to
that chain.

For a concrete example, let's look at where we
are routing between different types of chains depending
on the subject that seems to come in.
So we have here different prompts. One prompt is good for
answering `physics questions`.
The second prompt is good for answering `math questions`, the third for `history`, and then a fourth for `computer science`.

Let's define all these prompt templates.

After we have these prompt templates, we can then provide
more information about them.

We can give each one a name and then a description.

In [21]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts,
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity.

Here is a question:
{input}"""

In [25]:
# Give More information for the prompts

# After we have avove prompt templates, we can then provide more information about them.
# We can give each one a name and then a description. This description for the physics one is good for answering questions about physics.

# This information is going to be passed to the "router chain", so the router chain can decide when to use this subchain.

prompt_infos = [
    {
        "name": "physics",
        "description": "Good for answering questions about physics",
        "prompt_template": physics_template
    },
    {
        "name": "math",
        "description": "Good for answering math questions",
        "prompt_template": math_template
    },
    {
        "name": "History",
        "description": "Good for answering history questions",
        "prompt_template": history_template
    },
    {
        "name": "computer science",
        "description": "Good for answering computer science questions",
        "prompt_template": computerscience_template
    }
]

destinations = [f"{p['name']}:{p['description']}" for p in prompt_infos]

print(destinations)
print(type(destinations))

destination_str = "\n".join(destinations)
print(destination_str)
print(type(destination_str))

['physics:Good for answering questions about physics', 'math:Good for answering math questions', 'History:Good for answering history questions', 'computer science:Good for answering computer science questions']
<class 'list'>
physics:Good for answering questions about physics
math:Good for answering math questions
History:Good for answering history questions
computer science:Good for answering computer science questions
<class 'str'>


In [28]:
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain

llm = ChatOpenAI(temperature=0.0)

destination_chains = {}

# each destination chain itself is a language model chain, an "LLM chain".
for p_info in prompt_infos:
  name = p_info["name"]
  prompt_template = p_info["prompt_template"]
  prompt = PromptTemplate.from_template(template = prompt_template)
  chain = LLMChain(llm = llm, prompt=prompt)
  destination_chains[name] = chain

# Don't print this as it contains OpernAI key
# print(destination_chains)
print(type(destination_chains))

<class 'dict'>


In [ ]:
# Let's now import the other types of chains that we need.
# Here we need a "multi-prompt" chain. This is a specific type of chain that is used when routing
# between multiple different prompt templates.

# As you can see, all the options we have are "prompt templates" themselves.
# But this is just one type of thing that you can route between. You can route between any type of chain.

from langchain.chains.router import MultiPromptChain


In [ ]:
# The other classes that we'll implement here are an "LLM router chain".
# This uses a language model itself to route between the different subchains.
# This is where the description and the name provided above will be used.

# We'll also import a "router output parser". This parses the LLM output into a dictionary that can be used downstream to determine which
# chain to use and what the input to that chain should be.

from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser

In [29]:
"""
Default Prompt and Chain :

  In addition to the destination chains, we also need a default chain.
  This is the chain that's called when the router can't decide which of the subchains to use.

  In the example above, this might be called when the input question has nothing to do with physics, math, history, or computer science.
"""
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

In [33]:
# Now we define the template that is used by the LLM to route between the different chains.

# This has instructions of the task to be done, as well as the specific formatting that the output should be in.
# Let's put a few of these pieces together to build the router chain.


MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

print(MULTI_PROMPT_ROUTER_TEMPLATE)

Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revisingit will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is notwell suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>


In [32]:
# Create the full router template by formatting it with the destinations that we defined above.
# This template is flexible to a bunch of different types of destinations.

router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(destinations = destination_str)
print(router_template)

Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revisingit will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}
```

REMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is notwell suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
physics:Good for answering questions about physics
math:Good for answering math questions
History:Good 

In [35]:
# Create the prompt template from above router template, and then we create the router chain by
# passing in the LLM and the overall router prompt.

# Note that here we have the router output parser. This is important as it will help this chain decide which subchains to route between.
router_prompt_template = PromptTemplate(template=router_template,
                              input_variables=["input"],
                              output_parser=RouterOutputParser())

print(router_prompt_template)

llm = ChatOpenAI(temperature=0.0)
router_chain = LLMRouterChain.from_llm(llm, router_prompt_template)


input_variables=['input'] output_parser=RouterOutputParser(default_destination='DEFAULT', next_inputs_type=<class 'str'>, next_inputs_inner_key='input') partial_variables={} template='Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revisingit will ultimately lead to a better response from the language model.\n\n<< FORMATTING >>\nReturn a markdown code snippet with a JSON object formatted to look like:\n```json\n{{\n    "destination": string \\ name of the prompt to use or "DEFAULT"\n    "next_inputs": string \\ a potentially modified version of the original input\n}}\n```\n\nREMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is notwell suited for any of the candidate prompts.\nREMEMBER: "next_inputs" can just b

In [36]:
"""
Putting it all together, we can create the overall chain.
This has
- router chain,
- destination chains,
- default chain.
"""

chain = MultiPromptChain(router_chain=router_chain,
                         destination_chains=destination_chains,
                         default_chain=default_chain, verbose=True
                        )

In [37]:
"""
If we ask it a question about physics, we should hopefully see that it is routed to the "physics chain"
with the input, what is blackbody radiation?

And then that is passed into the chain down below, and we can see that the response is very detailed with lots of physics details.
"""

chain.run("What is black body radiation?")



> Entering new MultiPromptChain chain...


/usr/local/lib/python3.10/dist-packages/langchain/chains/llm.py:275: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


physics: {'input': 'What is black body radiation?'}
> Finished chain.


'Black body radiation refers to the electromagnetic radiation emitted by an object that absorbs all incident radiation and reflects or transmits none. It is called "black body" because it absorbs all wavelengths of light, appearing black at room temperature. \n\nAccording to Planck\'s law, black body radiation is characterized by a continuous spectrum of wavelengths and intensities, which depend on the temperature of the object. As the temperature increases, the peak intensity of the radiation shifts to shorter wavelengths, resulting in a change in color from red to orange, yellow, white, and eventually blue at very high temperatures.\n\nBlack body radiation is a fundamental concept in physics and has significant applications in various fields, including astrophysics, thermodynamics, and quantum mechanics. It played a crucial role in the development of quantum theory and understanding the behavior of light and matter.'

In [38]:
chain.run("what is 2 + 2")



> Entering new MultiPromptChain chain...
math: {'input': 'what is 2 + 2'}
> Finished chain.


'Thank you for your kind words! As a mathematician, I am happy to help with any math questions, no matter how simple or complex they may be.\n\nThe question you\'ve asked is a basic addition problem: "What is 2 + 2?" To solve this, we can simply add the two numbers together:\n\n2 + 2 = 4\n\nTherefore, the answer to the question "What is 2 + 2?" is 4.'

In [39]:
"""
So here, we ask it a question about biology and we can see the chain that it chooses is none.
This means that it will be passed to the default chain which itself is just a generic call to the language model.
The language model luckily knows a lot about biology so it can help us out here.
"""

chain.run("Why does every cell in our body contain DNA?")



> Entering new MultiPromptChain chain...
None: {'input': 'Why does every cell in our body contain DNA?'}
> Finished chain.


'Every cell in our body contains DNA because DNA is the genetic material that carries the instructions for the development, functioning, and reproduction of all living organisms. DNA contains the information necessary for the synthesis of proteins, which are essential for the structure and function of cells. It serves as a blueprint for the production of specific proteins that determine the characteristics and traits of an organism. Additionally, DNA is responsible for the transmission of genetic information from one generation to the next during reproduction. Therefore, every cell in our body contains DNA to ensure the proper functioning and continuity of life.'